<a href="https://colab.research.google.com/github/SerasLain/NLP-2019/blob/master/Pirozhki_classification%5BSidorova%2C_Belyaeva%2C_Salamatina%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Сбор датасета из БД

# Определение автора краткой поэзии
### выполнили А. Сидорова, А. Беляева, А. Саламатина

In [0]:
# Соединение с БД
import sqlite3

conn = sqlite3.connect('peraw.db')
c = conn.cursor()

In [0]:
import pandas as pd

Posts = pd.read_sql('SELECT * FROM Posts', conn)
Authors = pd.read_sql('SELECT * FROM Authors', conn)

DatabaseError: ignored

In [0]:
print(Authors.shape, Posts.shape)

(1541, 3) (17108, 6)


In [0]:
Posts.head()

,Post_id,Author_id,date,text,age,city
0,234497,Unknown,05,Лучшие [club95488043|депрессяшки] за апрель201...,40,Unknown
1,244615,Дей,06,заходишь в чат к хорошим людям\nсидят по добро...,40,Saint Petersburg
2,244396,Сыр,06,у нас в сизо неплохо кормят\nа иногда по выход...,40,Minsk
3,244348,Nomad,06,оксана под олегом в сексе\nтак извивалась что ...,42,Vologda
4,244316,pofiger,06,олегу денег не хватает\nни на непал ни на тибе...,51,Ramat Gan


In [0]:
Authors.head()

,Author_id,age,city
0,unknown,Unknown,Unknown
1,Дей,Unknown,Saint Petersburg
2,Сыр,Unknown,Minsk
3,Nomad,42,Vologda
4,pofiger,51,Ramat Gan


In [0]:
auth = {}
for row_obj in Authors.iterrows():
  row = row_obj[1]
  auth[row['Author_id']] = (row['age'], row['city'])

In [0]:
ages = []
cities = []
for author in Posts['Author_id']:
  try:
    age, city = auth[author]
  except KeyError:
    age = 'Unknown'
    city = 'Unknown'
  ages.append(age)
  cities.append(city)
Posts['age'] = ages
Posts['city'] = cities

### Обработка датасета

Какие есть проблемы:

- неизвестные возраста
- неизвестные города
- дата в unixtime

In [0]:
Posts.head()

,Post_id,Author_id,date,text,age,city
0,234497,Unknown,05,Лучшие [club95488043|депрессяшки] за апрель201...,40,Unknown
1,244615,Дей,06,заходишь в чат к хорошим людям\nсидят по добро...,40,Saint Petersburg
2,244396,Сыр,06,у нас в сизо неплохо кормят\nа иногда по выход...,40,Minsk
3,244348,Nomad,06,оксана под олегом в сексе\nтак извивалась что ...,42,Vologda
4,244316,pofiger,06,олегу денег не хватает\nни на непал ни на тибе...,51,Ramat Gan


In [0]:
Posts.to_csv('Posts.csv')

In [0]:
# Извлекаем из даты месяц
from datetime import datetime
ts = int("1284101485")

# if you encounter a "year is out of range" error the timestamp
# may be in milliseconds, try `ts /= 1000` in that case
dates = Posts['date']

new_date = []
for i in dates:
  a = datetime.utcfromtimestamp(int(i)).strftime('%m')
  new_date.append(a)
Posts['date'] = new_date

In [0]:
n = Posts['Author_id'][0]
Posts['Author_id'] = Posts['Author_id'].replace(n, 'Unknown')

In [0]:
# Заменяем неизвестный возраст на средний
import numpy as np
ages = [int(a) for a in Authors['age'] if a != 'Unknown']
mean_age = np.mean(ages)

In [0]:
mean_age = int(mean_age)

In [0]:
Posts['age'] = Posts['age'].replace('Unknown', mean_age)

In [0]:
Authors['city']

0                 Unknown
1        Saint Petersburg
2                   Minsk
3                 Vologda
4               Ramat Gan
5                 Unknown
6                Zarechny
7                 Unknown
8           New York City
9                  Moscow
10                 Moscow
11               Zlatoust
12                   Omsk
13       Saint Petersburg
14                Unknown
15                Unknown
16       Saint Petersburg
17              Krasnodar
18       Saint Petersburg
19                 Moscow
20       Saint Petersburg
21               Murmansk
22       Saint Petersburg
23                Unknown
24                 Moscow
25               Murmansk
26                Unknown
27                   Tula
28              Amsterdam
29                Saratov
              ...        
1511                Kazan
1512              Kharkiv
1513     Saint Petersburg
1514              Unknown
1515          Novosibirsk
1516               Moscow
1517              Unknown
1518        

### Tf-idf


In [0]:
import re
posts = []
for i in Posts['text']:
    i = re.sub('\\[.*?\\]', '', i)
    i = re.sub('\n', ' ', i)
    i = re.sub('\s+', ' ', i).lower()
    posts.append(re.sub(r'©.*', '', i))

In [114]:
Text1 = ''
for i in posts:
    Text1 = Text1 + ' ' + i
Text1[:300]

' лучшие за апрель2019 радуется ольга что пришёл апрель достаёт из шкафа платье икс икс эль  заходишь в чат к хорошим людям сидят по доброму молчат зайдёшь к плохим у них веселье порнуха водка тролли срач  у нас в сизо неплохо кормят а иногда по выходным из ресторанчика напротив приносит запах шаурмы'

In [0]:
from collections import defaultdict
def get_fdict(string):
    corp = defaultdict(int)
    word_arr = string.split(' ')
    corp_l = len(word_arr)
    for i in word_arr:
        if i != '':
            corp[i] += 1
    for k in corp:
        corp[k] = corp[k]/corp_l
    return corp

In [0]:
corp = get_fdict(Text1)

In [0]:
def get_tfidf(posts, corp):
    arr = []
    for post in posts:
        tf_dict = get_fdict(post)
        tfidf_dict = {}
        for k in tf_dict:
            if k != '':
                tfidf_dict[k] = tf_dict[k]*(1/corp[k])
        d = sorted(tfidf_dict.items(), key=lambda i: i[1])[-3:]
        arr.append(d)
    return arr

NameError: ignored

In [0]:
arr = get_tfidf(posts, corp)
arr_kw = []
for kw in arr:
    words = []
    for w in kw:
        words.append(w[0])
    arr_kw.append(' '.join(words))

In [120]:
print(arr_kw[:20])

['эль икс апрель2019', 'тролли заходишь порнуха', 'шаурмы сизо ресторанчика', 'червей извивалась накопал', 'непал тибет бобруйск', 'справа склонились престолов', 'поступать взломали продолжают', 'мерзнешь укрыли холодней', 'подсвечник слабеющих слабеющие', 'тусклый солёной камбалы', 'проснёмся сумрак семдесят', 'расставанье драмынбас печальную', 'заштопал душевные крестиком', 'дырочкой коронный сланцы', 'фильмов комиков президентов', 'заметила тонкий намёк', 'какое знать хуже', 'плюс пометкой цифру', 'рельсах маслу берлиоз', 'топтали туве бикинь']


In [0]:
Posts['keywords']= arr_kw

In [0]:
Posts['text'] = posts

In [124]:
Posts.head()

,Post_id,Author_id,date,text,age,city,keywords
0,234497,Unknown,05,лучшие за апрель2019 радуется ольга что пришёл...,40,Unknown,эль икс апрель2019
1,244615,Дей,06,заходишь в чат к хорошим людям сидят по добром...,40,Saint Petersburg,тролли заходишь порнуха
2,244396,Сыр,06,у нас в сизо неплохо кормят а иногда по выходн...,40,Minsk,шаурмы сизо ресторанчика
3,244348,Nomad,06,оксана под олегом в сексе так извивалась что о...,42,Vologda,червей извивалась накопал
4,244316,pofiger,06,олегу денег не хватает ни на непал ни на тибет...,51,Ramat Gan,непал тибет бобруйск


In [0]:
Posts.to_csv('Posts_kw.csv', encoding='utf-8')

###Word2Doc

In [0]:
#почистим наши посты
import re
posts = []
for i in Posts['text']:
    i = re.sub('\n', ' ', i)
    i = re.sub('    ', ' ', i)
    i = re.sub('  ', ' ', i)
    posts.append(re.sub(r'©.*', '', i))
    
#простемируем их
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
normal_sent = []
for i in posts:
    normal_words = []
    for word in i.split():
        p = morph.parse(word)[0]
        normal_words.append(p.normal_form)
    normal_sent.append(normal_words)
    
sents = []
for i in normal_sent:
    sent = ' '.join(i)
    sents.append(sent)

In [0]:
#теперь натренируем на этом всем модель doc2vec

tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(sents)]

max_epochs = 100
vec_size = 20
alpha = 0.025

model = Doc2Vec(size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1)
  
model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.iter)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

model.save("d2v.model")
print("Model Saved")